# Example of MQTT Publisher
We will use multiple implementations of message queues including <b>RabbitMQ/pika</b> and <b>MQTT</b>. This notebook is an illustration of publishing or sending a stream using <i>MQTT</i>. See https://pypi.org/project/paho-mqtt/ and a very helpful tutorial: http://www.steves-internet-guide.com/publishing-messages-mqtt-client/
<br>
<br>
Here is the most basic code to connect a sending stream to an MQTT publisher and an MQTT subsciber to a receiving stream. You can optimize the connection using ideas in the above tutorial.
<br>
<br>
The publisher and subscriber communicate through a specified topic and host. In this example, the data consists of a list of lists, and each sublist of data is published separately after which the sender pauses. In this notebook, <i>host</i> is 'localhost'. You should specify your own host computer.
<br>
<br>
Note: Run the notebook <i>ExampleMQTT_Subscriber</i> first. Then run this notebook: <i>ExampleMQTT_Publisher</i>. You should see the data that you published using this notebook printed in the notebook <i>ExampleMQTT_Subscriber</i>.

In [1]:
import time
import json
import threading

import sys
sys.path.append("../")
from IoTPy.agent_types.sink import sink_element
from IoTPy.core.stream import Stream

In [2]:
def on_connect(client, userdata, flags, rc):
    if rc==0: client.connected_flag=True
    else: print("Bad connection! Returned code = ", rc)

In [3]:
class MQTT_Publisher(object):
    def __init__(self, client_id='test_client', topic='topic', host='localhost'):
        self.client_id = client_id
        self.topic = topic
        self.host = host
        self.client = mqtt.Client(client_id)
        self.client.connected_flag = False
        self.client.connect(host)
        self.time_waiting_for_connection = 1.0
        self.n_tries_for_connection = 5
        self.client.on_connect = on_connect
        self.client.loop_start()
        # Wait for the connection to succeed.
        n_tries = 0
        while not self.client.connected_flag:
            time.sleep(self.time_waiting_for_connection)
            n_tries += 1
            if n_tries > self.n_tries_for_connection:
                break
        if n_tries > self.n_tries_for_connection:
            print ('Unsuccessful connecting')
        print ('Publisher connected!')
    def close(self):
        self.client.loop_stop()
        self.client.disconnect()
    def publish(self, stream):
        def publish_element(element):
            # rc, mid are return code, message id.
            rc, mid = self.client.publish(self.topic, json.dumps(element), qos=1)
        sink_element(func=publish_element, in_stream=stream, name='MQTT Agent')

In [4]:
from IoTPy.helper_functions.print_stream import print_stream
from IoTPy.core.stream import Stream, run
from IoTPy.concurrency.MQTT_Publisher import MQTT_Publisher
# multicore imports
from IoTPy.concurrency.multicore import get_processes_and_procs
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import extend_stream

In [5]:
def mqtt_publisher_test(DATA_TO_PUBLISH):
    # Step 0: Define agent functions and source threads
    # Step 0.0: Define agent functions.
    # This is the agent for the process called mqtt_publish_process.
    def mqtt_publish_agent(in_streams, out_streams):
        MQTT_Publisher(topic='topic', host='localhost').publish(in_streams[0])

    # Step 0.1: Define source threads.
    # This thread puts the data into the stream called 'mqtt_publish_stream'
    # and then terminates the stream.
    def thread_target_source(procs):
        for sublist in DATA_TO_PUBLISH:
            extend_stream(procs, sublist, stream_name='mqtt_publish_stream')
            time.sleep(0.1)
        # Finished. So, put '_finished' in the source stream and terminate it.
        extend_stream(procs, data=['_finished'], stream_name='mqtt_publish_stream')
        terminate_stream(procs, stream_name='mqtt_publish_stream')

    # Step 1: multicore_specification of streams and processes.
    # Specify Streams: list of pairs (stream_name, stream_type).
    # Specify Processes: name, agent function, 
    #       lists of inputs and outputs and sources, and additional arguments.
    multicore_specification = [
        # Streams
        [('mqtt_publish_stream', 'x')],
        # Processes
        [{'name': 'mqtt_publish_process', 'agent': mqtt_publish_agent,
          'inputs':['mqtt_publish_stream'], 'sources': ['mqtt_publish_stream']}]]

    # Step 2: Create processes.
    processes, procs = get_processes_and_procs(multicore_specification)

    # Step 3: Create threads (if any)
    thread_0 = threading.Thread(target=thread_target_source, args=(procs,))

    # Step 4: Specify which process each thread runs in.
    # thread_0 runs in the process called 'p1'
    procs['mqtt_publish_process'].threads = [thread_0]

    # Step 5: Start, join and terminate processes.
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

In [6]:
DATA_TO_PUBLISH = [['a', 0], [['c', 'd'], {'e':0, 'f':1}], [list(range(4))]]
mqtt_publisher_test(DATA_TO_PUBLISH)

Publisher connected!
